# Polyjuice Potion

You might be asking what is this...

Just a mix and mash of ADC, T2W and so on

TODO: Definire classe 1 e 2 per esempio classe 1: ISUP 2,3 mentre classe 2: ISUP 3,4.

Una classe puo avere 1 o piu valori

# Imports

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Data Reader (!IMPORTANT!)

In [14]:
def is_single_string(param):
    return isinstance(param, str)

def is_list_of_strings(param):
    return isinstance(param, (list, tuple)) and all(isinstance(item, str) for item in param)

In [15]:
def no_polyjuice_getter(sequence, labels, dataset):
    # Filter the dataset to include only AI annotations and selected sequences and ROI lesion
    filtered_dataset = dataset[dataset['annotator'] == 'AI']
    filtered_dataset = filtered_dataset[filtered_dataset['sequence'] == sequence]
    filtered_dataset = filtered_dataset[filtered_dataset['ROI'] == 'lesion']
    filtered_dataset.drop(columns=['annotator', 'sequence', 'ROI_ID', 'ROI','img_path', 'seg_path', 'extraction_ID'], inplace=True)
    dataset = filtered_dataset

    # Rename columns to match the labels dataset
    labels.rename(columns={'patient_id': 'patient_ID', 'study_id': 'study_ID'}, inplace=True)

    # We only want Magnetic Resonace Biopsy (MRBx) labels because Systematic Biopsy (SBx) labels are not for our usecase
    # we also remove those that have both because clicinians result might be biased
    #labels = labels[labels['histopath_type'] == 'MRBx']

    # Remove useless columns from labels dataset
    labels.drop(columns=['mri_date', 'histopath_type', 'center', 'lesion_ISUP', 'lesion_GS'], inplace=True)

    # Rename Yes to 1 and No to 0 in the labels dataset
    labels.case_csPCa = labels.case_csPCa.map(lambda x: 1 if x == 'YES' else 0)

    # Drop NA
    labels.dropna(inplace=True)

    # Remove High Correlation
    corr_matrix = dataset.drop(columns=['study_ID', 'patient_ID']).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
    dataset.drop(to_drop, axis=1, inplace=True)

    merge = pd.merge(dataset, labels, on=['patient_ID', 'study_ID'], how='inner')
    merge.drop(columns=['patient_ID', 'study_ID'], inplace=True)

    isup = 'case_ISUP'
    pca = 'case_csPCa'

    merge['ISUP>=3'] = merge[isup].map(lambda x: 1 if x >= 3 else 0)
    return merge

In [22]:
def polyjuice_getter(sequence, labels, dataset):
    polyjuice_dataset = None
    filtered_dataset = dataset[dataset['annotator'] == 'AI']
    filtered_dataset = filtered_dataset[filtered_dataset['ROI'] == 'lesion']

    labels.rename(columns={'patient_id': 'patient_ID', 'study_id': 'study_ID'}, inplace=True)
    labels.drop(columns=['mri_date', 'histopath_type', 'center', 'lesion_ISUP', 'lesion_GS'], inplace=True)
    labels.dropna(inplace=True)

    for index, s in enumerate(sequence):
        s_dataset = (filtered_dataset[filtered_dataset['sequence'] == s]).copy()
        for col in s_dataset.columns:
            if col != 'patient_ID' and col != 'study_ID':
                s_dataset.rename(columns={col : str(col + s)}, inplace=True)
        if index == 0:
            polyjuice_dataset = s_dataset
        else:
            polyjuice_dataset = pd.merge(polyjuice_dataset, s_dataset, on=['patient_ID', 'study_ID'])

    corr_matrix = dataset.drop(columns=['study_ID', 'patient_ID']).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
    dataset.drop(to_drop, axis=1, inplace=True)

    merge = pd.merge(dataset, labels, on=['patient_ID', 'study_ID'], how='inner')
    merge.drop(columns=['patient_ID', 'study_ID'], inplace=True)

    isup = 'case_ISUP'
    pca = 'case_csPCa'

    merge['ISUP>=3'] = merge[isup].map(lambda x: 1 if x >= 3 else 0)
    return merge
        


In [20]:
def get_data(sequence):

    is_polyjuice = None

    if is_list_of_strings(sequence):
        # E' un vettore
        if len(sequence) > 1:
            # Ha lungezza > 1 quindi siamo in un polisucco (mix)
            is_polyjuice = True
        else:
            is_polyjuice = False
    else:
        is_polyjuice = False



    labels = pd.read_csv('marksheet.csv')
    dataset = pd.read_csv('PI-CAI_features')

    # Classic and easy flow
    if is_polyjuice == False:
        return no_polyjuice_getter(sequence, labels, dataset)
    else:
        return polyjuice_getter(sequence, labels, dataset)
        
        

In [23]:
get_data(['t2w', 'adc'])

ValueError: could not convert string to float: '10000_1000000_t2w_prostate_AI'